In [1]:
import optics_tools as ot
from optics_tools import const as const

The optics_tools library is for calculation of various optical system properties of a satellite imager. Units are mostly SI, but in some cases the units are indicated by the variable name. Examples to watch out for are those involving a finite number of particles, e.g. readout noise. 

Validate equations using formulas from book Larson 'Space missiongs analysis and design' page 290

In [2]:
# Setup computation - these are all variables

ALTITUDE = 600e3
BITS = 12
DARK_SIGNAL_EPS = 0 #1e3 # electrons/pixel/second
F_NUMBER = 3.9
FOCAL_LENGTH = 0.3
FRAME_RATE = 30
GSD = 20
INTEGRATION_TIME = 24.1E-6
OPTICAL_TRANSMISSION_FACTOR = 0.75
PIXELS_ALONG = 1
PIXELS_CROSS = 1024
#PIXELS_TOTAL = PIXELS_ALONG * PIXELS_CROSS
PIXEL_DIM_CROSS = 20e-6
RADIANCE_INTEGRATED = 40 * 0.3 # W/m^2/sr/um x 100nm
QUANTUM_EFFICIENCY = 0.5
READ_OUT_NOISE_EP= 25 # electrons/pixel
WAVELENGTH = 600e-9

# Perform calcs

In [3]:
GSD = ot.calc_gsd(ALTITUDE, FOCAL_LENGTH, PIXEL_DIM_CROSS)
GSD

40.00000000000001

In [4]:
SWATH = ot.calc_swath_at_nadir(GSD, PIXELS_CROSS)
SWATH

40960.00000000001

In [5]:
INTEGRATION_TIME = GSD/ot.calc_velocity_ground_track(ot.calc_earth_orbital_period(ALTITUDE))  
INTEGRATION_TIME

0.005790360262958956

In [6]:
APERTURE_DIAMETER = ot.calc_aperture_diameter(F_NUMBER, FOCAL_LENGTH)
APERTURE_DIAMETER

0.07692307692307693

In [7]:
SENSOR_INCIDENT_POWER = 4.3e-11 # W, from book

In [8]:
INCIDENT_POWER_AT_PIXEL = ot.calc_pixel_incident_power(SENSOR_INCIDENT_POWER, OPTICAL_TRANSMISSION_FACTOR)
INCIDENT_POWER_AT_PIXEL

3.225e-11

In [9]:
energy_imparted = ot.calc_energy_per_integration(INCIDENT_POWER_AT_PIXEL, INTEGRATION_TIME)
signal_photons = ot.calc_photons_from_energy(energy_imparted, WAVELENGTH)
signal_photons

564039

In [10]:
#signal_photons = ot.calc_signal_photons(ALTITUDE, APERTURE_DIAMETER, GSD, INTEGRATION_TIME, OPTICAL_TRANSMISSION_FACTOR, RADIANCE_INTEGRATED, WAVELENGTH)
signal_electrons = signal_photons * QUANTUM_EFFICIENCY
signal_electrons

282019.5

In [11]:
signal_electrons_shott = ot.calc_shott_noise(signal_electrons)
#dark_electrons = DARK_SIGNAL_EPS * INTEGRATION_TIME
#dark_electrons_shott = ot.calc_shott_noise(dark_electrons)
readout_noise = READ_OUT_NOISE_EP
#total_noise = ot.calc_uncorrellated_noise(signal_electrons_shott, dark_electrons, dark_electrons_shott, readout_noise)
total_noise = ot.calc_uncorrellated_noise(signal_electrons_shott, readout_noise)

total_noise

531.5881864752075

In [12]:
SNR = int(signal_electrons / total_noise)
print("The system SNR is {}".format(SNR))

The system SNR is 530
